# SLM and LLM token Prompt complexity and intent detection processing

## Imports

In [ ]:
import ollama
import common

client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

def read_file(file:str):
    with open(file, 'r') as f:
        return f.read()
        
file_401k = read_file('data/401k.txt')
file_benefits = read_file('data/benefits.txt')

## Call Ollama and Azure GPT

In [ ]:
def gpt_completion(prompt:str, model='gpt4o',temperature=0.1) -> str:
    completion = client.chat.completions.create(
        model=model,
        messages=[
        {
           "role": "user",
            "content": prompt,
        }],
        temperature=temperature
    )
    
    return completion.choices[0].message.content

    
def ollama_completion(prompt:str, model='llava',temperature=0.1) -> int:
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': prompt,        
    },    
    ],options = {'temperature': temperature})
    
    return response['message']['content']


## Determine intent and prompt complexity and process prompt on depending on basis

In [ ]:
def determine_complexity(prompt:str):
    template ="""system:
You are an assistant that can score the complexity of a user's question or statement. A score can be from 0 to 10, where 0 is the least complex and 10 is the most complex.

user:
Score the following question or statement from 0 to 10:

<PROMPT>

Respond with the score only.
"""
    res = ollama_completion(template.replace('<PROMPT>', prompt))
    # is res a number?
    try:
        return int(res)
    except:
        return 0



def intent_detection(prompt:str):
    template ="""system:
You are an assistant that can determine intent from the following list of intents:

401k: a user asks a 401k.
benefits: a user asks a company benefits.
weather: a user asks a questions about the weather.
other: a user asks a question about anything else.

user:
<PROMPT>

Respond with the intent word only unless it is the weather intent then respond with the intent and the location. For example: weather, New York.
"""

    return ollama_completion(template.replace('<PROMPT>', prompt))

def process_for_complexity(prompt: str):
    score = determine_complexity(prompt)
    print("Score:",score)
    if score > 5:
        # GPT processing
        print("GPT processing")
        print(gpt_completion(prompt))
    else:
        # Ollama processing
        print("Ollama processing")
        print(ollama_completion(prompt))


def process_for_intent(prompt: str):
    intent = intent_detection(prompt).strip()
    print("Intent:",intent)
    if intent == 'weather':
        location = intent.split(',')[1].strip()
        print(f"Intent is weather for {location}. You need a weather service.")
    elif intent == '401k':
        print("GPT processing")
        print(gpt_completion(f"{prompt}\nContent:\nfile_401k\nRespond in one senetence. Use only the provided content."))
    elif intent == 'benefits':
        print("Ollama processing")
        print(ollama_completion(f"{prompt}\nContent:\nfile_benefits\nRespond in one sentence. Use only the provided content."))
    else:
        print("Ollama processing")
        print(ollama_completion(prompt))

## Intent processing

In [ ]:
process_for_intent("What is the speed of light?")
process_for_intent("What is a 401k account?")
process_for_intent("What are some company benefits?")
process_for_intent("What is the weather in London?")

## Complexity processing

In [ ]:
process_for_complexity("What is the speed of light?")
process_for_complexity("""Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>""")
